###Target:
Reach 99.4% accuracy consistently in last 3-4 epochs by a applying more regularization in the form of dropout and increase LR rate to understand if convergence occurs faster and use LR schedular to change LR wherever Loss gets to plateau

###Results:
1. Parameters: 7,917
2. Best Train Accuracy: 99.08%
3. Best Test Accuracy: 99.43% 
4. greater than 99.4% in last 4 epochs(12-15)

###Analysis:
1. Dropout added the necessary regularization required to make model robust
2. Though increased LR was helping in fast convergence but after 5-6 epochs it would tapper off implying a need for smaller LR so used a step LR schedule at after 4 epochs and decreased it to 0.8 times the previous LR

###All the changes added to the base CNN architechture to acheive the target accuracy 
1. Input image normalization
2. Image augmentation - rotation for Train data 
3. Batch Normalization
4. Dropouts 
5. LR scheduler 
6. GAP

 

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
#Receptive field is calcualted based on actual formula
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #block1
        self.conv1 = nn.Conv2d(1, 8, 3,padding = True) #o/p size:28; rf: 3
        self.Batchnorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 15, 3) #o/p size: 26; rf: 5
        self.Batchnorm2 = nn.BatchNorm2d(15)


        #transition block
        self.pool1 = nn.MaxPool2d(2, 2) #o/p size: 13; rf: 6
        self.pool1trns = nn.Conv2d(15, 10, 1)#o/p size: 13; rf: 6
        self.Batchnormtrns1 = nn.BatchNorm2d(10)
        
        #block2
        self.conv3 = nn.Conv2d(10,14,3) #o/p size: 11; rf: 10
        self.Batchnorm3 = nn.BatchNorm2d(14)
        self.dp3 = nn.Dropout(p = 0.10) 
        self.conv4 = nn.Conv2d(14, 16, 3) #o/p size: 9; rf: 14
        self.Batchnorm4 = nn.BatchNorm2d(16)
        self.dp4 = nn.Dropout(p = 0.10)
        self.conv5 = nn.Conv2d(16, 20, 3) #o/p size: 7; rf: 18
        self.Batchnorm5 = nn.BatchNorm2d(20)
        self.dp5 = nn.Dropout(p = 0.10)

        #gap and 1X1 
        self.conv6_avgp = nn.AvgPool2d(kernel_size=7) #o/p size: 1; rf: 30
        self.pool2trns = nn.Conv2d(20,10,1) #6


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.Batchnorm1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.Batchnorm2(x)


        x = self.pool1(x)
        x = self.pool1trns(x)
        x = F.relu(x)
        x = self.Batchnormtrns1(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.Batchnorm3(x)
        x = self.dp3(x)
 
        x = self.conv4(x)
        x = F.relu(x)
        x = self.Batchnorm4(x)
        x = self.dp4(x)

        x = self.conv5(x)
        x = F.relu(x)
        x = self.Batchnorm5(x)
        x = self.dp5(x)

        x = self.conv6_avgp(x)

        x = self.pool2trns(x)


        x = x.view(-1, 10)
        return F.log_softmax(x,dim=-1)

In [ ]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 15, 26, 26]           1,095
       BatchNorm2d-4           [-1, 15, 26, 26]              30
         MaxPool2d-5           [-1, 15, 13, 13]               0
            Conv2d-6           [-1, 10, 13, 13]             160
       BatchNorm2d-7           [-1, 10, 13, 13]              20
            Conv2d-8           [-1, 14, 11, 11]           1,274
       BatchNorm2d-9           [-1, 14, 11, 11]              28
          Dropout-10           [-1, 14, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           2,032
      BatchNorm2d-12             [-1, 16, 9, 9]              32
          Dropout-13             [-1, 16, 9, 9]               0
           Conv2d-14             [-1, 2

In [ ]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-8.0, 8.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    train_loss = 0 
    train_correct = 0
    #scheduler.step()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(test_loader.dataset)
    print('Epoch: {:.0f},LR: {}.\nTrain set: train Average loss: {:.4f}, train_Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch,optimizer.param_groups[0]['lr'],train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
        

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred_test = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred_test.eq(target.view_as(pred_test)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    print('Test set: test Average loss: {:.4f}, test Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
from torch.optim.lr_scheduler import StepLR

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
#4,0.8
scheduler = StepLR(optimizer, step_size=4, gamma=0.8)

for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

Epoch: 1,LR: 0.03.
Train set: train Average loss: 1.4490, train_Accuracy: 55762/60000 (92.9367%)

Test set: test Average loss: 0.0552, test Accuracy: 9834/10000 (98.3400%)

Epoch: 2,LR: 0.03.
Train set: train Average loss: 0.4469, train_Accuracy: 58641/60000 (97.7350%)

Test set: test Average loss: 0.0444, test Accuracy: 9853/10000 (98.5300%)

Epoch: 3,LR: 0.03.
Train set: train Average loss: 0.3709, train_Accuracy: 58844/60000 (98.0733%)

Test set: test Average loss: 0.0333, test Accuracy: 9890/10000 (98.9000%)

Epoch: 4,LR: 0.03.
Train set: train Average loss: 0.3307, train_Accuracy: 58995/60000 (98.3250%)

Test set: test Average loss: 0.0288, test Accuracy: 9902/10000 (99.0200%)

Epoch: 5,LR: 0.024.
Train set: train Average loss: 0.2939, train_Accuracy: 59083/60000 (98.4717%)

Test set: test Average loss: 0.0248, test Accuracy: 9915/10000 (99.1500%)

Epoch: 6,LR: 0.024.
Train set: train Average loss: 0.2552, train_Accuracy: 59207/60000 (98.6783%)

Test set: test Average loss: 0.0232